# Formatage pour tableau

## formatage des fichiers en csv

In [16]:
import json
import csv
import re
from urllib.parse import urlparse

# Chemins des fichiers JSON et CSV
product_links_json = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\1scraping\datasets\products\product_links.json"
data_fusion_json = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\2traitement\datasets\data_fusion_cleaned.json"
product_links_csv = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\product_links.csv"
data_fusion_products_csv = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\data_fusion_products.csv"
data_fusion_comments_csv = r"C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\data_fusion_comments.csv"

def extraire_categorie(url):
    """
    Extraire le nom lisible de la catégorie depuis une URL.

    Args:
        url (str): URL de la catégorie.

    Returns:
        str: Nom lisible de la catégorie.
    """
    return urlparse(url).path.strip("/").split("/")[-1].replace("-", " ").capitalize()

def convertir_product_links_en_csv(product_links_json, output_csv):
    """
    Convertir le fichier JSON des liens produits en un fichier CSV.

    Args:
        product_links_json (str): Chemin du fichier JSON des liens produits.
        output_csv (str): Chemin du fichier CSV de sortie pour les liens produits.

    Returns:
        None
    """
    try:
        print("Conversion de product_links.json en CSV...")
        with open(product_links_json, "r", encoding="utf-8") as file:
            product_links_data = json.load(file)

        with open(output_csv, "w", encoding="utf-8", newline="") as outfile:
            fieldnames = ["Categorie", "Lien_Produit", "Nom_Produit", "Prix", "Avis"]
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for categorie, produits in product_links_data.items():
                categorie_nom = extraire_categorie(categorie)
                for produit in produits:
                    writer.writerow({
                        "Categorie": categorie_nom,
                        "Lien_Produit": produit.get("link"),
                        "Nom_Produit": produit.get("name"),
                        "Prix": produit.get("price"),
                        "Avis": produit.get("reviews"),
                    })

        print(f"Fichier CSV généré avec succès : {output_csv}")

    except Exception as e:
        print(f"Une erreur s'est produite lors de la conversion de product_links : {e}")

def convertir_data_fusion_en_csv(data_fusion_json, product_links_json, products_output_csv, comments_output_csv):
    """
    Convertir le fichier JSON des données fusionnées en deux fichiers CSV :
    - Un pour les produits.
    - Un pour les commentaires.

    Args:
        data_fusion_json (str): Chemin du fichier JSON des données fusionnées.
        product_links_json (str): Chemin du fichier JSON des liens produits pour associer les catégories.
        products_output_csv (str): Chemin du fichier CSV de sortie pour les produits.
        comments_output_csv (str): Chemin du fichier CSV de sortie pour les commentaires.

    Returns:
        None
    """
    try:
        print("Conversion de data_fusion_cleaned.json en CSV...")
        # Charger les données JSON
        with open(data_fusion_json, "r", encoding="utf-8") as file:
            data_fusion_data = json.load(file)

        with open(product_links_json, "r", encoding="utf-8") as file:
            product_links_data = json.load(file)

        # Associer les produits aux catégories
        produit_categorie = {}
        for categorie, produits in product_links_data.items():
            for produit in produits:
                produit_categorie[produit["link"]] = extraire_categorie(categorie)

        # Écrire les données produits
        with open(products_output_csv, "w", encoding="utf-8", newline="") as products_file:
            products_fieldnames = [
                "Categorie", "Lien_Produit", "Nom_Produit", "Prix", "Note_Moyenne", "Nombre_Avis", "Nombre_Commentaires"
            ]
            products_writer = csv.DictWriter(products_file, fieldnames=products_fieldnames)
            products_writer.writeheader()

            for produit in data_fusion_data:
                categorie = produit_categorie.get(produit["link"], "Non catégorisé")
                products_writer.writerow({
                    "Categorie": categorie,
                    "Lien_Produit": produit.get("link"),
                    "Nom_Produit": produit.get("name"),
                    "Prix": produit.get("price"),
                    "Note_Moyenne": produit.get("average_rating"),
                    "Nombre_Avis": produit.get("total_reviews"),
                    "Nombre_Commentaires": len(produit.get("comments", [])),
                })

        # Écrire les données commentaires
        with open(comments_output_csv, "w", encoding="utf-8", newline="") as comments_file:
            comments_fieldnames = [
                "Categorie", "Lien_Produit", "Nom_Produit", "Titre_Commentaire", "Commentaire",
                "Titre_Commentaire_Processe", "Commentaire_Processe", "Contient_Contraste"
            ]
            comments_writer = csv.DictWriter(comments_file, fieldnames=comments_fieldnames)
            comments_writer.writeheader()

            for produit in data_fusion_data:
                categorie = produit_categorie.get(produit["link"], "Non catégorisé")
                for commentaire in produit.get("comments", []):
                    comments_writer.writerow({
                        "Categorie": categorie,
                        "Lien_Produit": produit.get("link"),
                        "Nom_Produit": produit.get("name"),
                        "Titre_Commentaire": commentaire.get("title"),
                        "Commentaire": commentaire.get("content"),
                        "Titre_Commentaire_Processe": commentaire.get("processed_title"),
                        "Commentaire_Processe": commentaire.get("processed_content"),
                        "Contient_Contraste": commentaire.get("contains_contrast"),
                    })

        print(f"Fichiers CSV générés avec succès : {products_output_csv}, {comments_output_csv}")

    except Exception as e:
        print(f"Une erreur s'est produite lors de la conversion de data_fusion_cleaned : {e}")

# Exécution des conversions
convertir_product_links_en_csv(product_links_json, product_links_csv)
convertir_data_fusion_en_csv(data_fusion_json, product_links_json, data_fusion_products_csv, data_fusion_comments_csv)


Conversion de product_links.json en CSV...
Fichier CSV généré avec succès : C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\product_links.csv
Conversion de data_fusion_cleaned.json en CSV...
Fichiers CSV générés avec succès : C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\data_fusion_products.csv, C:\TOUS_MES_FICHIERS\ProjetPython\TravauxTechniques\4formatage_pour_tableau\data_fusion_comments.csv


In [17]:
import pandas as pd
from collections import Counter

# Chemins des fichiers
input_file = r"data_fusion_comments.csv"  # Fichier source
output_file = r"unigrammes_par_categorie.csv"  # Fichier de sortie

# Charger les données
data = pd.read_csv(input_file)

# Combiner les colonnes de commentaires traités
data["Combined_Comments"] = data["Titre_Commentaire_Processe"].fillna("") + " " + data["Commentaire_Processe"].fillna("")

# Groupement par catégorie et extraction des mots
unigram_data = []

for categorie, group in data.groupby("Categorie"):
    words = []
    for comment in group["Combined_Comments"]:
        words.extend(comment.split())  # Diviser les mots dans chaque commentaire
    word_counts = Counter(words)  # Compter les mots pour chaque catégorie
    for word, frequency in word_counts.items():
        unigram_data.append({"Categorie": categorie, "Word": word, "Frequency": frequency})

# Créer un DataFrame et sauvegarder dans un fichier CSV
unigram_df = pd.DataFrame(unigram_data)
unigram_df.to_csv(output_file, index=False, encoding="utf-8")

print(f"Fichier CSV généré avec succès : {output_file}")


Fichier CSV généré avec succès : unigrammes_par_categorie.csv


In [18]:
import pandas as pd
from collections import Counter
from itertools import tee

# Chemins des fichiers
input_file = r"data_fusion_comments.csv"  # Fichier source
output_file = r"bigrams_par_categorie.csv"  # Fichier de sortie

def generate_bigrams(words):
    """
    Générer des bigrammes à partir d'une liste de mots.

    Args:
        words (list): Liste de mots.

    Returns:
        list: Liste de tuples contenant les bigrammes.
    """
    a, b = tee(words)
    next(b, None)
    return list(zip(a, b))

# Charger les données
data = pd.read_csv(input_file)

# Combiner les colonnes de commentaires traités
data["Combined_Comments"] = data["Titre_Commentaire_Processe"].fillna("") + " " + data["Commentaire_Processe"].fillna("")

# Groupement par catégorie et extraction des bigrammes
bigram_data = []

for categorie, group in data.groupby("Categorie"):
    bigrams = []
    for comment in group["Combined_Comments"]:
        words = comment.split()  # Diviser les mots dans chaque commentaire
        bigrams.extend(generate_bigrams(words))  # Générer les bigrammes
    bigram_counts = Counter(bigrams)  # Compter les bigrammes pour chaque catégorie
    for bigram, frequency in bigram_counts.items():
        bigram_data.append({"Categorie": categorie, "Bigram": " ".join(bigram), "Frequency": frequency})

# Créer un DataFrame et sauvegarder dans un fichier CSV
bigram_df = pd.DataFrame(bigram_data)
bigram_df.to_csv(output_file, index=False, encoding="utf-8")

print(f"Fichier CSV généré avec succès : {output_file}")


Fichier CSV généré avec succès : bigrams_par_categorie.csv
